In [1]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
import traceback
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
driver = None

In [3]:
def open_main_screen():
    global driver
    path = "C:\\Program Files (x86)\\chromedriver.exe"
    service = Service(path)
    options = Options()
    #options.add_argument("--headless")
    chrome_options = Options()
    chrome_options.add_argument("--disable-notifications")  # Bildirimleri devre dışı bırak
    chrome_options.add_argument("--disable-popup-blocking") # Pop-up'ları engelle
    chrome_options.add_argument("--start-maximized")       # Tam ekran başlat
    
    # Tarayıcı profili
    prefs = {"profile.default_content_setting_values.notifications": 2}  # Bildirim izinlerini kapat
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.trendyol.com/"
    driver.get(url)
    
    try:
        cookies_button_rejected = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
        cookies_button_rejected.click()
    except Exception as e:
        print('cookies not found')
        print(e)


In [140]:
keys = [
    'searched_key_word',
    'brand',
    'title',
    'find_dim',
    'find_dim_from_content',
    'guess_dimensions',
    'guess_dim_from_comment',
    'image_path',
]

df = pd.DataFrame(columns=keys)

In [5]:
def search_items(entry):
    global driver
    search_box = driver.find_element(By.XPATH,'//*[@id="sfx-discovery-search-suggestions"]/div/div/input')
    search_box.clear()
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  

In [6]:
def select_most_reated():
    global driver
    recommended = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/div/div')
    recommended.click()
    most_rated = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/ul/li[7]/span')
    most_rated.click()

In [7]:
def find_dim():
    global driver
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/section/div/div/div/ul') 
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        if texts_with_numbers == []:
            texts_with_numbers = [text.strip() for text in relevant_details if re.search(r"(\d+)\s*(cm|mm)", text)]
        return texts_with_numbers
    except Exception as e:
        return f"An error occurred: {str(e)}" # every product must contain these field so if it's not return me a error.

In [8]:
def find_dim_from_content():
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim','ebat']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH, '//*[@id="urun-aciklamasi"]/div')
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        if texts_with_numbers == []:
            texts_with_numbers = [text.strip() for text in relevant_details if re.search(r"(\d+)\s*(cm|mm)", text)]
        return texts_with_numbers
    except Exception as e:
        return None #every product don't have to contain these field so if it's not, note return me any thing

In [9]:
def guess_dimensions():
    global driver
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-reviews-btn')
    except:
        return 'there is no comment'
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(1)
    #driver.execute_script("window.scrollBy(0, -400);") bu biraz daha kesin olmayan yol
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
    except:
        try:
            button = driver.find_element(By.XPATH, "//button[contains(@class, 'tags-wrapper__button') and not(contains(@class, 'rotate-180'))]")
            button.click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
        except:
            return None

    time.sleep(1)
    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer
            
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r"(\d+)\s*(cm|mm)", answer):
            return answer

    return None

In [10]:
def guess_dim_from_comment(entry = 'Boyut' ):
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-questions-btn')
    except:
        return 'there is no comment'
    element.click()
    time.sleep(2)
    search_box = driver.find_element(By.XPATH,'//*[@id="questions-and-answers-app"]/div/div/div/div/div/div[3]/div[2]/div/div[1]/input')
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)

    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer

    for answer in answers:
       answer = answer.text.split('\n')[1]
       answer = answer.lower()
       if re.search(r"(\d+)\s*(cm|mm)", answer):
           return answer
    return None
    

In [11]:
def click_all_features():
    global driver
    ozellikler = driver.find_element(By.CLASS_NAME,'button-all-features')
    ozellikler.click()

In [12]:
def close_understood():
    global is_it_first_time
    if is_it_first_time:
        global driver
        understood = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/div[2]/div/div[2]/div[2]/div/div[1]/aside/div[1]/div/div[2]/div/div[2]/div/div/button')
        understood.click()
        is_it_first_time = False

In [144]:
def close_understood2():
    global is_it_first_time2
    if is_it_first_time2:
        global driver
        understood = driver.find_element(By.XPATH,'//*[@id="rating-and-review-app"]/div/div/div/div[3]/div/div/div[3]/div[1]/div/div[2]/div[2]/div[1]/button')
        understood.click()
        is_it_first_time2 = False

In [13]:
def scrap_brand():
    global driver
    brand = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'product-brand-name')]")))
    return brand.text

In [14]:
def scrap_title():
    all_title = driver.find_element(By.CLASS_NAME,'pr-new-br').text
    brand = scrap_brand()
    title  = all_title.strip(brand).strip()
    return brand, title

In [145]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

def scrape_photo(photo_number):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "rvw-cnt-tx"))).click()
    #enter_images = driver.find_element(By.XPATH, "//span[@class='ps-tags__tag--bold' and contains(text(), 'fotoğraflı')]")
    time.sleep(2)
    
    if is_it_first_time2 == True:
        close_understood2()
        
    try:
        enter_images = driver.find_element(By.XPATH,'//*[@id="rating-and-review-app"]/div/div/div/div[3]/div/div/div[3]/div[1]/div/div[1]/button')
        enter_images.click()
    except:
        return "There is no enough photo"

   
    gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
    image_links = []
    #for index in range(0,(int(photo_number/20))): # there are 20 photos in scroll panel when we open that
    for index in range(0,photo_number):
        if index % 20==0 and index !=0:
            gallery = driver.find_element(By.CLASS_NAME, "image-gallery-content")
            gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
            driver.execute_script("arguments[0].scrollTop += 400;", gallery)
            time.sleep(1)
            driver.execute_script("arguments[0].scrollTop += 400;", gallery)
            time.sleep(1)
            driver.execute_script("arguments[0].scrollTop += 100;", gallery)
            time.sleep(3)
            
    gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
    for item in gallery_items:
        img = item.find_element(By.TAG_NAME, "img")
        image_links.append(img.get_attribute('src'))

    return image_links

In [42]:
gallery_items = driver.find_elements(By.CLASS_NAME, "image-gallery-item")
image_links = []
for index, item in enumerate(gallery_items, start=1):
    img = item.find_element(By.TAG_NAME, "img")
    image_links.append(img.get_attribute('src'))

len(image_links)

60

In [21]:
driver.execute_script("window.scrollBy(0, 400);")

In [90]:
from collections import Counter

url_counts = Counter(images)
duplicates = [url for url, count in url_counts.items() if count > 1]
len(duplicates) #there is no dublicat url of photos.

0

In [ ]:
images

In [ ]:
windows = driver.window_handles
len(windows)

In [ ]:
driver.back()

In [20]:
windows = driver.window_handles
driver.switch_to.window(windows[1])
len(windows)

2

In [ ]:
windows = driver.window_handles
driver.switch_to.window(windows[1])
gallery = driver.find_element(By.CLASS_NAME, "image-gallery")
driver.execute_script("arguments[0].scrollTop += 300;", gallery) 
#driver.execute_script("window.scrollBy(0, 400);")

In [155]:
def run():
    global driver
    global is_it_first_time
    is_it_first_time = True
    global is_it_first_time2
    is_it_first_time2 = True
    open_main_screen()
    search_keys = ['oyuncak bebek']
    for key in search_keys:
        search_items(key)
        select_most_reated()
        time.sleep(2)
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'variant-options-wrapper')))
        for i in products:
            i.click()
            time.sleep(2)
            windows = driver.window_handles
            driver.switch_to.window(windows[1])
            if is_it_first_time:
                close_understood()
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            time.sleep(2)
            brand = scrap_title()[0]
            title = scrap_title()[1]
            key_word = key
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            click_all_features()
            time.sleep(1)
            find_dim_ = find_dim()
            driver.execute_script("window.scrollBy(0, 400);") #bazi urunlerde aciklama olmadigi icin ortala yerine kaydiriyorum.
            find_dim_from_content_ = find_dim_from_content()
            guess_dimensions_ = guess_dimensions()
            if guess_dimensions_ == 'there is no comment':
                guess_dimensions_ = None
            else:
                driver.back()
            guess_dim_from_comment_ = guess_dim_from_comment()
            
            if guess_dim_from_comment_ == 'there is no comment':
                guess_dim_from_comment_ = None
            else:
                driver.back()
            driver.execute_script("window.scrollTo(0, 0);")
            #buradan sonra fotograf cekme asamalri gelmesi lazim
            images = scrape_photo(60)
            print(f'cekilen fotograf sayisi: {len(images)}')
            #buradan once fotograf cekme asamalri gelmesi lazim
            for img in images:
                df.loc[len(df)] = [key_word, brand ,title, find_dim_, find_dim_from_content_, guess_dimensions_,
                                   guess_dim_from_comment_, img]
                print(df.tail(1))
                
            driver.close()
            driver.switch_to.window(windows[0])
            driver.execute_script("window.scrollTo(0, 0);")
            break

In [156]:
run()

cekilen fotograf sayisi: 60
    searched_key_word        brand  \
120     oyuncak bebek  Global Toys   

                                                    title find_dim  \
120  Mavi Beyaz Ve Pembe Renk Ağlayan Ve Konuşan Et Bebek       []   

                                    find_dim_from_content  \
120  [30 cm boyunda, 20 cm genişliğinde sevimli tasarım.]   

                      guess_dimensions  \
120  merhabalar, ürün boyutu 30cm dir.   

                              guess_dim_from_comment  \
120  merhaba, 30x20 cm'dir. sağlıklı günler dileriz.   

                                                                                                                  image_path  
120  https://review-rating.mncdn.com/mnresize/140/210/prod/img/2016931/2019922/2019922/341493644/14196636-1677856711781.jpeg  
    searched_key_word        brand  \
121     oyuncak bebek  Global Toys   

                                                    title find_dim  \
121  Mavi Beyaz Ve Pembe Renk Ağ

In [154]:
pd.set_option('display.max_colwidth', None)
df

,searched_key_word,brand,title,find_dim,find_dim_from_content,guess_dimensions,guess_dim_from_comment,image_path
0,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2017928/2025904/2025904/202437657/8188178-1723150691660.jpeg
1,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2017928/2020919/2036871/202437657/15991354-1710860591357.jpeg
2,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2017928/2025904/2021916/202437657/43920348-1722771461430.jpeg
3,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2017928/2025904/2042853/202437657/103263687-1724584838219.jpeg
4,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2017928/2026901/2039862/202437657/26553260-1727018448945.jpeg
...,...,...,...,...,...,...,...,...
115,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2016931/2019922/2037868/202437657/89621020-1679420120681.jpeg
116,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2015934/2019922/2025904/202437657/14335630-1649594589923.jpeg
117,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2015934/2026901/2034877/202437657/7006876-1668844563817.jpeg
118,cop kovasi,Okyanus Home,Krom Yedekli Wc Kağıtlık - Beyaz Çöp Kovası Ve Wc Fırça Seti,[],None,"merhaba, çöp kovası ölçüleri 22,3x18 cm’dir.","merhaba 19,5x22 cmdir",https://review-rating.mncdn.com/mnresize/140/210/prod/img/2015934/2026901/2034877/202437657/7006876-1668844564381.jpeg


In [ ]:
windows = driver.window_handles
driver.switch_to.window(windows[1])
driver.execute_script("window.scrollBy(0, 400);")

In [ ]:
guess_dimensions()

In [ ]:
guess_dim_from_comment()

In [133]:
windows = driver.window_handles
driver.switch_to.window(windows[1])

In [136]:
close_understood2()

bu fonksiyona girildi
False
